<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Fine_tuning_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Yelp review modeling

We will use the `yelp_review_full` dataset and create a model that takes the review in and give me a 5-star rating.

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/p

Load the dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Show dataset structure

In [ ]:
dataset

Show an example

In [ ]:
dataset["train"][100]

##Step1:

Tokenize the dataset

In [ ]:
from transformers import AutoTokenizer

# Create the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") # https://huggingface.co/transformers/v3.3.1/pretrained_models.html

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Get a small subset of the dataset
sampled_dataset_train = dataset["train"].shuffle(seed=42).select(range(1000))
sampled_dataset_test = dataset["test"].shuffle(seed=42).select(range(1000))

# Tokenize the dataset
tokenized_dataset_train = sampled_dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_test = sampled_dataset_test.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset_train

##Step 2:

Train the model

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments

#Create the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Setup a metric to evaluate the model

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Train!

In [ ]:
from transformers import TrainingArguments, Trainer

# Setup the trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="steps", eval_steps=75)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    compute_metrics=compute_metrics,
)

trainer.train()

## Step 3:

Save the model

In [ ]:
trainer.save_model("./yelp_classifier")

##Step 4: Things to play with:

- Train from scratch.
- Hyper-parameter tuning.
- Study the dataset.
- Try a different model.
- Why is there no train logging?
- Load the model.
- Test the model.
